In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import date
from matplotlib import pyplot as plt 
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
from matplotlib.dates import DateFormatter
import seaborn as sns
import tweepy
from twitter_authentication import bearer_token
import contractions
import numpy as np
from transformers import pipeline
from IPython.display import clear_output
from scipy import stats
from scipy.stats import chi2_contingency
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
from umap import UMAP

## Tweepy scrape

In [ ]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)
start_time = ['2022-05-01T00:00:00Z']
end_time = ['2022-06-13T23:59:59Z']

In [ ]:
tweets = []

for i,j in zip(start_time,end_time):
    print(i)
    print(j)
    for response in tweepy.Paginator(client.search_all_tweets, 
                                     query = 'กัญชา -is:retweet lang:th',
                                     #query = 'cannabis thailand OR weed thailand OR marijuana thailand OR pot thailand OR blunt thailand OR mary jane thailand -is:retweet lang:en',
                                     user_fields = ['username', 'public_metrics', 'description', 'location'],
                                     tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                     expansions = ['author_id', 'geo.place_id'],
                                     start_time = i,
                                     end_time = j):
                                     
        time.sleep(1)
        tweets.append(response)

    result = []
    user_dict = {}
    for response in tweets:
        for user in response.includes['users']:
            user_dict[user.id] = {'username': user.username, 
                                  'followers': user.public_metrics['followers_count'],
                                  'tweets': user.public_metrics['tweet_count'],
                                  'description': user.description,
                                  'location': user.location
                                 }
        for tweet in response.data:
            author_info = user_dict[tweet.author_id]
            result.append({'author_id': tweet.author_id,
                           'geo.place_id': tweet.geo,
                           'username': author_info['username'],
                           'author_followers': author_info['followers'],
                           'author_tweets': author_info['tweets'],
                           'author_description': author_info['description'],
                           'author_location': author_info['location'],
                           'tweet': tweet.text,
                           'created_at': tweet.created_at,
                           'retweets': tweet.public_metrics['retweet_count'],
                           'replies': tweet.public_metrics['reply_count'],
                           'likes': tweet.public_metrics['like_count'],
                           'quote_count': tweet.public_metrics['quote_count'],
                            })
    df = pd.DataFrame(result)    

In [ ]:
df.to_csv('file_name.csv', index =False)

## Pre-processing data

In [ ]:
df = pd.read_csv('file_name.csv')

In [ ]:
#using with eng
df['processed']= df['tweet'].replace(r'http\S+', ' ', regex=True)
df['processed']=df['processed'].apply(contractions.fix)
df['processed']=df['processed'].apply(lambda x :[t for t in x.split() if not t.startswith('@')] ).apply(lambda x: ' '.join(x))
df['processed']= df['processed'].replace(r'[^A-Za-zก-๏]+', ' ', regex=True)
df['processed']= df['processed'].replace(r'[ก-๙]+', ' ', regex=True)
df['processed']=df['processed'].apply(lambda x : x.strip())
df['processed']=df['processed'].str.lower()

In [ ]:
#using with eng after translate
df['tr_processed']= df['translated'].replace(r'http\S+', ' ', regex=True)
df['tr_processed']=df['tr_processed'].apply(contractions.fix)
df['tr_processed']=df['tr_processed'].apply(lambda x :[t for t in x.split() if not t.startswith('@')] ).apply(lambda x: ' '.join(x))
df['tr_processed']= df['tr_processed'].replace(r'[^A-Za-zก-๏]+', ' ', regex=True)
df['tr_processed']= df['tr_processed'].replace(r'[ก-๙]+', ' ', regex=True)
df['tr_processed']=df['tr_processed'].apply(lambda x : x.strip())
df['tr_processed']=df['tr_processed'].str.lower()

In [ ]:
#using with thai
df['processed']= df['tweet'].replace(r'http\S+', ' ', regex=True)
df['processed']=df['processed'].apply(contractions.fix)
df['processed']=df['processed'].apply(lambda x :[t for t in x.split() if not t.startswith('@')] ).apply(lambda x: ' '.join(x))
df['processed']= df['processed'].replace(r'[^A-Za-zก-๏]+', ' ', regex=True)
df['processed']= df['processed'].replace(r'[ๆ]+', ' ',regex=True)
df['processed']=df['processed'].apply(lambda x : x.strip())
df['processed']=df['processed'].str.lower()

In [ ]:
#remove duplicate tweets
df_unique = df.drop_duplicates(['choose_target_column'])

In [ ]:
df_unique.to_csv('file_name.csv', index=False)

## Translation-Thai --> English

In [ ]:
df = pd.read_csv('file_name.csv')

In [ ]:
from IPython.display import clear_output
from deep_translator import GoogleTranslator
tweet_translated =[]
for n in range(round(len(df))):
    print(n)
    clear_output(wait = True)
    to_translate = df.processed[n]
    translated = GoogleTranslator(source='auto', target='en').translate(to_translate)
    tweet_translated.append(translated)

In [ ]:
df['translated'] = tweet_translated
df.to_csv('file_name.csv', index=False)

## Sentiment analysis

In [ ]:
df = pd.read_csv('file_name.csv')

In [ ]:
df['processed'] = df['processed'].astype(str)

In [ ]:
text = df.processed[0]

In [ ]:
sentimentAnalysis = pipeline("sentiment-analysis")
print(sentimentAnalysis(text))

In [ ]:
sentiment = []
score = []
for n in range(len(df)):
    print(n)
    clear_output(wait = True)
    text = df.processed[n]
    sentimentAnalysis = pipeline("sentiment-analysis")
    s = sentimentAnalysis(text)
    s_sen = s[0]['label']
    s_score = s[0]['score']
    sentiment.append(s_sen)
    score.append(s_score)

In [ ]:
df['sentiment_transformer'] = sentiment
df['transformer_score'] = score

In [ ]:
df['new_sentiment'] = df.sentiment_transformer
df.loc[df.transformer_score < 0.99, 'new_sentiment'] = 'NEUTRAL'

In [ ]:
df.to_csv('file_name.csv', index=False)

## Sort date

In [ ]:
df = pd.read_csv('file_name.csv')

In [ ]:
#Get before 2022-06-09 (1May2022-8June2022)
df['created_at'] = pd.to_datetime(df['created_at'])
mask1 = (df['created_at'] > '2022-04-30T23:59:59Z') & (df['created_at'] <= '2022-06-08T23:59:59Z')
df.loc[mask1]
df_sort1 = df.loc[mask1]
df_sort1.to_csv('file_name.csv')

In [ ]:
#Get since 2022-06-09 (9June2022-13June2022)
df['created_at'] = pd.to_datetime(df['created_at'])
mask1 = (df['created_at'] > '2022-06-08T23:59:59Z') & (df['created_at'] <= '2022-06-13T23:59:59Z')
df.loc[mask1]
df_sort1 = df.loc[mask1]
df_sort1.to_csv('file_name.csv')

## Visualization

### Visualize number of tweets per day

In [ ]:
df = pd.read_csv('file_name.csv')

In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'])
df_count = df['created_at'].groupby(df['created_at'].dt.date).count()
df_count.columns = ['created_at','count']

In [ ]:
df_count.to_csv('file_name.csv', header =['count'], index=True)

In [ ]:
data = pd.read_csv("file_name.csv")

In [ ]:
data['created_at'] = pd.to_datetime(data['created_at']).dt.date
plt.plot(data['created_at'],data['count'],color='black')
plt.setp(plt.gca().xaxis.get_majorticklabels(),rotation=90)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%b-%Y'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=3))
plt.xlim([datetime.date(2022, 5, 1), datetime.date(2022, 6, 15)])
plt.savefig('file_name.tiff', dpi=300, format='tiff', bbox_inches='tight')

### Visualize sentiment distribution

In [ ]:
BF = pd.read_csv("file_name.csv")
BF = BF.groupby("new_sentiment").sum()
#BF.drop(['Unnamed: 0'], axis=1, inplace = True)
BF = BF.reset_index()
labels = BF['new_sentiment']
count = BF['count']
colors = ("#CDCDCD", "#E5E5E5", "#A3A3A3") 
fig1, ax1 = plt.subplots()
ax1.pie(count, labels=labels, autopct='%1.1f%%', textprops={'size': 'x-large'}, colors=colors)
ax1.axis('equal')
plt.title('title', pad=30)
plt.savefig('file_name.tiff', dpi=300, format='tiff', bbox_inches='tight')

### Stack bar chart

In [ ]:
df = pd.read_csv('file_name.csv')

In [ ]:
#set new index
df.set_index("Category", inplace = True)

In [ ]:
df = df.div(df.sum(axis=1), axis=0)
df = round(df*100)

In [ ]:
df = df.sort_values("Category")

In [ ]:
# create stacked bar chart
ax = df.plot(kind='bar', stacked=True, cmap="tab20b")

# labels for x & y axis
plt.xlabel('')
plt.ylabel('ylabel')

#reverse legend
handles, labels = ax.get_legend_handles_labels()
plt.legend(reversed(handles),reversed(labels), loc="best", ncol=1, bbox_to_anchor=(1, 0.5, 1, 0.5))
plt.xticks(rotation=0)

# title of plot
plt.title('name_of_title')
plt.savefig('file_name.tiff', dpi=300, format='tiff',bbox_inches='tight')

## Statistic calculation

In [ ]:
BF = pd.read_csv("file_name.csv")
BF = BF.groupby("new_sentiment").sum()
BF.drop(['Unnamed: 0'], axis=1, inplace = True)
BF = BF.reset_index()
labels = BF['new_sentiment']
cal = BF['count'].sum(axis = 0)
print(cal)
BF.to_csv('file_name.csv', index = False)

In [ ]:
BF = pd.read_csv("file_name.csv")
AF = pd.read_csv("file_name.csv")

In [ ]:
BF_T = BF.T
BF_T = BF_T.rename(columns=BF_T.iloc[0]).drop(BF_T.index[0])
AF_T = AF.T
AF_T = AF_T.rename(columns=AF_T.iloc[0]).drop(AF_T.index[0])

In [ ]:
chi = pd.concat([BF_T,AF_T])
c, p, dof, expected = chi2_contingency(chi) 
stat = (c,p,dof)
chi_sq = pd.DataFrame([stat], columns =['chi-square','p-value','degrerOfreedom'])

In [ ]:
chi_sq.to_csv('file_name.csv', index = False)

## Topic modeling

In [ ]:
df = pd.read_csv('file_name.csv')
doc = list(df.processed)

In [ ]:
topic_model = BERTopic()
umap_model = UMAP(n_neighbors=15, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)
topic_model = BERTopic(umap_model=umap_model, top_n_words = 15)
topics, probs = topic_model.fit_transform(doc)

In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv('file_name.csv',index=True)

In [ ]:
topic_info_result = topic_model.get_topics()
topic_info_result_df = pd.DataFrame.from_dict(topic_info_result)
topic_info_result_df.to_csv('file_name.csv',index=True)

In [ ]:
topic_model.get_representative_docs()

In [ ]:
topic_table = topic_model.get_representative_docs()
topic_tab = pd.DataFrame(topic_table)
topic_tab.columns = ['tweet']

In [ ]:
topic_tab.to_csv('file_name.csv', index=False)